## <b>IMPORTAÇÃO DE BIBLIOTECAS

In [2]:
import cv2
import numpy as np
from scipy.spatial import distance
import dlib
import pandas
import math
import os
import matplotlib.pyplot as plt

## <b>CÁLCULO DOS LANDMARKS FACIAIS

In [3]:
def getPoints(path_land_pred, path_png, option):
    #1º PASSO (PRÉ-PROCESSAMENTO): CARREGAMOS A IMAGEM, APLICAMOS A ESCALA DE CINZA E DEFINIMOS O DETECTOR FACIAL E O DE LANDMARKS 
    img = cv2.imread(path_png)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(img_gray)
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(path_land_pred)
    faces = detector(img_gray)
    #2º PASSO (PROCESSAMENTO): APLICAMOS O DETECTOR DE LANDMARKS, E DEFINIMOS OS PONTOS X E Y DOS 81 PONTOS FACIAIS
    for face in faces:
        landmarks = predictor(img_gray, face)
        landmarks_points = []
        val_x = []
        val_y = []
        for n in range(0, 81):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_points.append((x, y))
            cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
            points = np.array(landmarks_points, np.int32)
    #3º PASSO (SAÍDA): RETORNAMOS EM 1 A IMAGEM COM OS LANDMARKS, EM 2 A IMAGEM INICIAL, OU O ARRAY DOS LANDMARKS
    if option == '1':
        cv2.imshow("Image 1", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imwrite('../Imagens/Ontario/result/test_woman2.jpg', img)
        
    elif option == '2':
        img_new = cv2.imread(path_png)
        cv2.imshow("Image 2", img_new)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        return landmarks_points

## <b>CRIAÇÃO DA MÁSCARA E COMPARAÇÃO

In [31]:
#1º PASSO (MÉDIA DOS PONTOS): CALCULAMOS A MÉDIAS DOS LANDMARKS DAS FACES QUE FORAM CARREGADAS
def calculo(ldmk_points):
    mean_matrix = []
    mean_matrix = np.matrix.round(sum(np.array(ldmk_points))/len(ldmk_points))
    mean_matrix = mean_matrix.astype('int')
                
    return mean_matrix

#2º PASSO (COMPARAÇÃO): FAZEMOS A COMPARAÇÃO ENTRE OS ROSTOS DE TESTE COM A MÁSCARA DE TREINO
def comparacao(mask, points):
    pts = []
    pts_comp = []
    
    for i in range(len(points)):
        pts = []
        for k in range(81):
            valor = distance.euclidean(points[i][k], mask[0][k])
            pts.append(valor)
        pts_comp.append(pts)
    for i in range(0, len(pts_comp)):
        print("mulher " + str(i+1) + " = " , abs(np.mean(pts_comp[i])))

        
def main():
    ldmk_points = []
    mask = []
    points = []
    pts_comparacao = []

    for i in range(1, len(os.listdir('../Imagens/Ontario/train/'))+1):
        ldmk_points.append(getPoints('../Landmarks/shape_predictor_81_face_landmarks.dat', '../Imagens/Ontario/train/teste' + str(i) + '.jpg', '0'))
    
    for i in range(1, len(os.listdir('../Imagens/Ontario/test/'))+1):
        points.append(np.array(getPoints('../Landmarks/shape_predictor_81_face_landmarks.dat', '../Imagens/Ontario/test/teste' + str(i) + '.jpg', '0')))

    
    mask.append((calculo(ldmk_points)))
    comparacao(mask, points)

In [32]:
if __name__ == "__main__":
    main()

mulher 1 =  23.514736970951834
mulher 2 =  19.646193701061776
mulher 3 =  12.32569558460885
mulher 4 =  20.184100315852362
mulher 5 =  14.01440174515263
